In [1]:
import xgboost
import shap
import matplotlib.pyplot as plt
import lightgbm as lgb
import pandas as pd
import numpy as np
import mlflow
import joblib
import pickle
from feature_engine.creation import RelativeFeatures
import uuid
from optbinning import BinningProcess
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import scipy
import statsmodels.api as sm
from scipy.interpolate import interp1d
from scipy.special import logit
from scipy.interpolate import UnivariateSpline



from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,f1_score,roc_auc_score,accuracy_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin


In [2]:
from helpers.evaluate import train_and_test,eval_metrics
from helpers.transformers import OptbinningScikit,LowessTransformer

In [3]:
#mlflow.create_experiment('logreg',artifact_location='models')

In [4]:
X,y = shap.datasets.adult()
X_display,y_display = shap.datasets.adult(display=True)

X_train,X_test,y_train,y_test=train_test_split(X_display,y_display,test_size=0.2,random_state=0)

features=X_display.columns.to_list()
cat_cols=X_display.select_dtypes(include='category').columns.to_list()
num_cols=[col for col in features if col not in cat_cols]

In [ ]:

logreg=LogisticRegression()


In [ ]:
train_and_test(logreg,X,y,'3',desc='logreg')

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


{'roc_auc': 0.8270419200913794, 'accuracy': 0.7979425763856901, 'recall': 0.3799373040752351, 'precision': 0.6495176848874598}


In [ ]:
opt_cat_logreg=Pipeline([('opt',
ColumnTransformer([
     
          ("text_preprocess", OptbinningScikit(),cat_cols)],
      remainder='passthrough',verbose_feature_names_out=False)),
      ('logreg',LogisticRegression())
      ])

In [ ]:
train_and_test(opt_cat_logreg,X_display,y_display,'3',desc='opt_cat_logreg')

{'roc_auc': 0.8703866291188024, 'accuracy': 0.8314140948871488, 'recall': 0.6037617554858934, 'precision': 0.6738978306508048}


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [3]:
opt_logreg=Pipeline([('opt', OptbinningScikit()),
      ('logreg',LogisticRegression())
      ])

In [12]:
train_and_test(opt_logreg,X_display,y_display,'3',desc='opt_cat_logreg')

{'roc_auc': 0.901796101838171, 'accuracy': 0.8456932289267619, 'recall': 0.5749216300940438, 'precision': 0.7371382636655949}


In [5]:
opt_cat_lowess_num_logreg=Pipeline([('opt',
ColumnTransformer([
     
          ("text_preprocess", OptbinningScikit(),cat_cols),
          ('num_process',LowessTransformer(fit_spline=False),num_cols)],
      remainder='passthrough',verbose_feature_names_out=False)),
      ('logreg',LogisticRegression())
      ])

In [6]:
train_and_test(opt_cat_lowess_num_logreg,X_display,y_display,'3',desc='opt_cat_lowess_num_logreg')

invalid value encountered in divide


{'roc_auc': 0.903282803494552, 'accuracy': 0.8496852448948258, 'recall': 0.5768025078369906, 'precision': 0.7516339869281046}


In [9]:
opt_cat_spline_num_logreg=Pipeline([('opt',
ColumnTransformer([
     
          ("text_preprocess", OptbinningScikit(),cat_cols),
          ('num_process',LowessTransformer(fit_spline=True),num_cols)],
      remainder='passthrough',verbose_feature_names_out=False)),
      ('logreg',LogisticRegression())
      ])

train_and_test(opt_cat_spline_num_logreg,X_display,y_display,'3',desc='opt_cat_spline_num_logreg')

invalid value encountered in divide


{'roc_auc': 0.9031384294913063, 'accuracy': 0.8504529402732995, 'recall': 0.587460815047022, 'precision': 0.74780526735834}


In [8]:
opt_cat_logit_spline_num_logreg=Pipeline([('opt',
ColumnTransformer([
     
          ("text_preprocess", OptbinningScikit(),cat_cols),
          ('num_process',LowessTransformer(use_logit=True,fit_spline=False),num_cols)],
      remainder='passthrough',verbose_feature_names_out=False)),
      ('logreg',LogisticRegression())
      ])

train_and_test(opt_cat_logit_spline_num_logreg,X_display,y_display,'3',desc='opt_cat_logit_spline_num_logreg')

invalid value encountered in divide


{'roc_auc': 0.903282803494552, 'accuracy': 0.8496852448948258, 'recall': 0.5768025078369906, 'precision': 0.7516339869281046}
